In [21]:
import pandas as pd
import re
from PyPDF2 import PdfReader
import os
import numpy as np

def extract_fund_info(content):
    """펀드 기본 정보 추출"""
    info = {}
    
    # 펀드명
    name_match = re.search(r"펀드명\s+(라이프[^\\n]+)", content)
    if name_match:
        info['펀드명'] = name_match.group(1).strip()
    
    # 운용규모
    nav_match = re.search(r"운용규모\s+\(NAV\)\s+약\s+([0-9,\.]+)억", content)
    if nav_match:
        info['운용규모'] = nav_match.group(1).strip() + "억"
    
    # 설정일
    date_match = re.search(r"설정일\s+(\d{4}년\s+\d{1}분기)", content)
    if date_match:
        info['설정일'] = date_match.group(1).strip()
        
    # 기준가
    price_match = re.search(r"기준가\s+([\d,\.]+)", content)
    if price_match:
        info['기준가'] = price_match.group(1).strip()
    
    return info

def extract_performance_metrics(content):
    """성과 지표 추출"""
    fund_name = re.search(r"펀드명\s+(라이프[^\\n]+)", content).group(1).strip()
    
    # 펀드별 지표 설정
    if "SPAC" in fund_name:
        metrics_order = [
            "설정일 이후 수익률",
            "YTD 수익률",
            "연율화 수익률",
            "연율화 수익률 (일 단위)",
            "연율화 변동성",
            "베타 (KOSDAQ대비)",
            "정보 비율 (KOSDAQ대비)",
            "MDD"
        ]
    else:
        metrics_order = [
            "설정일 이후 수익률",
            "YTD 수익률",
            "연율화 수익률",
            "연율화 수익률 (일 단위)",
            "연율화 변동성",
            "베타 (KOSPI대비)",
            "정보 비율 (KOSPI대비)",
            "MDD"
        ]

    pattern = r"성과 지표 I\s+2024-09-30\s+기준(.*?)업/다운"
    match = re.search(pattern, content, re.DOTALL)
    if not match:
        return pd.DataFrame()
    
    table_content = match.group(1).strip()
    rows = [row.strip() for row in table_content.split('\n') if row.strip()]
    
    data = []
    for row in rows:
        cols = [col.strip() for col in re.split(r'\s+', row) if col.strip()]
        if len(cols) >= 4:
            try:
                metric_name = ' '.join(cols[:-4]) if len(cols) > 4 else cols[0]
                values = [float(val.replace('+', '')) if val != '-' else 0.0 for val in cols[-4:]]
                
                for metric in metrics_order:
                    if metric.lower() in metric_name.lower():
                        data.append([metric] + values)
                        break
            except ValueError:
                continue

    df = pd.DataFrame(data, columns=['지표명', 'Fund', 'KOSPI', 'KOSPI200', 'KOSDAQ'])
    df.set_index('지표명', inplace=True)
    
    return df

def process_pdfs(pdf_paths):
    """PDF 파일들 처리"""
    fund_data = {}
    
    for pdf_path in pdf_paths:
        try:
            reader = PdfReader(pdf_path)
            content = ""
            for page in reader.pages:
                content += page.extract_text()
            
            fund_info = extract_fund_info(content)
            perf_df = extract_performance_metrics(content)
            
            if fund_info and not perf_df.empty:
                print(f"\n{'='*50}")
                print(f"펀드명: {fund_info['펀드명']}")
                print(f"운용규모: {fund_info['운용규모']}")
                print(f"설정일: {fund_info['설정일']}")
                print(f"기준가: {fund_info['기준가']}")
                print(f"\n[성과 지표]")
                print(perf_df)
                print('='*50)
                
                fund_data[fund_info['펀드명']] = (fund_info, perf_df)
                
        except Exception as e:
            print(f"Error processing {pdf_path}: {str(e)}")
    
    return fund_data

# 실행 코드
pdf_files = [
    '/Users/yujihun/Desktop/캡스톤/예제/RG-NP-000018-라이프 SPAC-KR-20240930.pdf',
    '/Users/yujihun/Desktop/캡스톤/예제/RG-NP-100004-라이프 주식형-KR-20240930.pdf',
    '/Users/yujihun/Desktop/캡스톤/예제/RG-NP-100030-라이프 채권혼합-KR-20240930.pdf'
]


# 판다스 출력 옵션 설정
pd.set_option('display.precision', 2)
pd.set_option('display.float_format', lambda x: f'{x:,.2f}')
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_rows', None)

# 파일 경로 설정 및 처리
current_dir = os.getcwd()
pdf_paths = [os.path.join(current_dir, pdf_file) for pdf_file in pdf_files]
fund_data = process_pdfs(pdf_paths)


펀드명: 라이프 SPAC
운용규모 (NAV) 약 80.0억
설정일 2021년 1분기
기준가 1,545.81 ('24.9.30 수정기준가 기준)
펀드 수익률 추이
기간별 누적수익률
2024-09-30 기준 Fu
운용규모: 80.0억
설정일: 2021년 1분기
기준가: 1,545.81

[성과 지표]
                  Fund  KOSPI  KOSPI200  KOSDAQ
지표명                                            
설정일 이후 수익률       54.58 -19.91    -13.45  -15.31
YTD 수익률           8.59 -11.85     -2.34   -3.70
연율화 수익률          13.15  -6.10     -4.01   -4.60
연율화 수익률          15.49  -5.65     -3.82   -4.35
연율화 변동성           5.54  19.15     13.95   14.78
베타 (KOSDAQ대비)     0.07   0.00      0.58    0.57
정보 비율 (KOSDAQ대비)  0.64   0.00      0.06    0.03
MDD              -6.80 -38.53    -34.79  -36.11

펀드명: 라이프 주식형
운용규모 (NAV) 약 2,000억
설정일 2021년 3분기
기준가 1,645.15 ('24.9.30 수정기준가 기준)
펀드 수익률 추이
기간별 누적수익률
2024-09-30 기준 Fu
운용규모: 2,000억
설정일: 2021년 3분기
기준가: 1,645.15

[성과 지표]
                  Fund  KOSPI  KOSPI200  KOSDAQ
지표명                                            
설정일 이후 수익률       64.52 -19.88    -19.66  -26.24
YTD 수익률          17.45  -2.34     -3.70